## Generate synthetic data using private marginals
---

#### Clean and get dataset and include important functions and packages

In [46]:
# get dataset
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

bear = pd.read_csv('https://raw.githubusercontent.com/jbennett979/Data_Privacy_FP/refs/heads/main/north_america_bear_killings.csv')

In [47]:
# Cleaning up the dataset
bear.columns = [c.lstrip() for c in bear]
# capitalizing columns name for age and gender
bear= bear.rename(columns={'age': 'Age'})
bear= bear.rename(columns={'gender': 'Gender'})

bear['Age'] = pd.to_numeric(bear['Age'], errors = 'coerce')
bear

,Name,Age,Gender,Date,Month,Year,Type,Location,Description,Type of bear,Hunter,Grizzly,Hikers,Only one killed
0,Erin Johnson,27,female,6/19/2017,6,2017,Wild,"Pogo mine, Alaska","Johnson, a contract employee for Pogo Mine, wa...",Black bear,0,0,0,1
1,Patrick Cooper,16,male,6/18/2017,6,2017,Wild,"Indian, Alaska",Cooper was chased and mauled by a bear while r...,Black bear,0,0,0,1
2,Daniel Ward O'Connor,27,male,5/10/2015,5,2015,Wild,"near Mackenzie, British Columbia",Ward was killed by a bear while he slept near ...,Black bear,0,0,0,1
3,Darsh Patel,22,male,9/21/2014,9,2014,Wild,"near West Milford, New Jersey",Patel was about to begin hiking with four frie...,Black bear,0,0,1,1
4,Lorna Weafer,36,female,5/7/2014,5,2014,Wild,"near Fort McMurray, Alberta","Weafer, a Suncor worker was attacked at the re...",Black bear,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,Baby Laird,1,NaN,10/5/1908,10,1908,Captive,"Tucson, Arizona",After a bear escaped from a cage at Elysian Gr...,Black bear,0,0,0,1
160,John Dicht,18,male,11/24/1906,11,1906,Wild,"Elk County, Pennsylvania","Thinking the bear was dead, Dicht began skinni...",Black bear,0,0,0,1
161,Mary Porterfield,3,female,5/19/1901,5,1901,Wild,"Job, West Virginia",The children were gathering flowers near their...,Black bear,0,0,0,0
162,Wilie Porterfield,5,male,5/19/1901,5,1901,Wild,"Job, West Virginia",The children were gathering flowers near their...,Black bear,0,0,0,0


#### Finding a differentially private average from the data set in order to safely compare outputs to the data

In [48]:
# Differentially private value to compare synthetic data against
def pick_b(epsilon, s):
    bs = range(0, 100, 10)
    last_result = 0
    epsilon_i = epsilon / len(bs)

    for b in bs:
        clipped_sum = s.clip(upper=b).sum()
        result = laplace_mech(clipped_sum, sensitivity=b, epsilon=epsilon_i)
        if result < last_result:
            return b
        else:
            last_result = result
    return bs[-1]

def dp_sum_age(epsilon):
    clipping_param = pick_b(epsilon, bear['Age'])
    # print("clipping parameter: ", clipping_param)
    age_dp = bear['Age'].clip(upper=clipping_param).sum()

    return laplace_mech(age_dp, sensitivity=clipping_param, epsilon=epsilon)

# compute a differentially private average of age column
def dp_avg_age(epsilon):
    dp_sum = dp_sum_age(epsilon/2)
    age_dp = bear['Age'].shape[0]
    dp_count = laplace_mech(age_dp, sensitivity=1, epsilon=epsilon/2)

    dp_avg = dp_sum/dp_count
    return dp_avg

# TODO: remove actual querying of dataset
print("real avg: ", bear['Age'].mean())
print("dp_avg: ", dp_avg_age(1.0))


real avg:  36.09756097560975
dp_avg:  28.882253491134396


#### Functions to generate differentially private marginals

In [49]:
# calculates a differentially private one-way marginal for a given column
def dp_marginal(col, epsilon):
    ### BEGIN SOLUTION
    hist = bear[col].value_counts()
    dp_hist = hist.apply(lambda x: laplace_mech(x, 1, epsilon))
    clipped = dp_hist.clip(lower=0)
    marginal = clipped / clipped.sum()
    return marginal

In [50]:
# builds a 2-way marginal with differential privacy.
def dp_two_marginal(col1, col2, epsilon):
    # make 2D histogram
    hist = bear[[col1, col2]].value_counts()
    dp_hist = hist.apply(lambda x: laplace_mech(x, sensitivity=1, epsilon=epsilon))
    syn_rep = dp_hist.clip(lower=0)

    marginal = syn_rep / syn_rep.sum()
    return marginal.to_frame(name='probability').reset_index()


* Start off by generating a synthetic representation of the age column
* From the age column, generate a two way marginal for age and something else
    * Type of bear
    * Hiker
    * Hunter
* Compare the accuracies


In [51]:
# Generate a two way synthetic rep for age and type of bear
def two_way_bear_hist(col1, col2, epsilon):
    # goal: 2-dimensional histogram
    hist = bear[[col1, col2]].value_counts()
    dp_hist = hist.apply(lambda x: laplace_mech(x, sensitivity=1, epsilon=epsilon))
    return dp_hist.clip(lower=0)

age_type = two_way_bear_hist('Age', 'Type of bear', 1.0)
print(age_type)
type_age = two_way_bear_hist('Type of bear', 'Age', 1.0)
print(type_age)

Age  Type of bear
40   Brown bear      8.720019
24   Black bear      6.613813
19   Brown bear      6.824625
3    Black bear      6.807082
38   Brown bear      2.870505
                       ...   
74   Black bear      0.123015
76   Brown bear      0.910995
77   Black bear      0.290510
     Brown bear      0.679439
93   Black bear      0.092444
Name: count, Length: 96, dtype: float64
Type of bear  Age
Brown bear    40     8.535237
Black bear    24     4.453792
Brown bear    19     4.692246
Black bear    3      1.907844
Brown bear    38     3.858571
                       ...   
Polar Bear    29     1.111167
              31     1.180871
              43     0.000000
              46     0.000000
              64     1.220381
Name: count, Length: 96, dtype: float64


In [52]:
def two_marginal(col1, col2, epsilon):
    # 1. generate the synthetic representation
    syn_rep = two_way_bear_hist(col1, col2, epsilon)
    # 2. normalize to get a probability distribution
    marginal = syn_rep / syn_rep.sum()
    return marginal.to_frame(name='probability').reset_index()

def gen_samples(n, marginal):
    samples = marginal.sample(n=n, replace=True, weights='probability')
    return samples.drop(columns='probability')

marginal = two_marginal('Type of bear','Age', 1.0)
gen_samples(15, marginal)

,Type of bear,Age
29,Black bear,16
18,Brown bear,53
47,Black bear,44
63,Black bear,68
94,Polar Bear,46
18,Brown bear,53
7,Black bear,1
8,Black bear,37
94,Polar Bear,46
80,Brown bear,59
